# Poker Hand Comparison Tool

This is an exercise that I did recently, and I wanted to share the notebook to my Github page so that it would hopefully be instructional to others.

Objectives:
- pass all test cases
- make code that is *modular*, *reusable*, and *easily maintained*
- explain my design choices

**Game Notes:**

I am assuming that people are familiar with the scoring rules for poker hands. For those who aren't, more information can be found here: https://www.cardplayer.com/rules-of-poker/hand-rankings

Also, A = 14, K = 13, Q = 12, J = 11, 10 = 10, ..., A = 1.

Finally, I am not differentiating between two flushes based on high card. It is trivially easy for the reader to add this function.

**Google Colab:** https://colab.research.google.com/github/jjc16/Project_Notebooks/blob/master/poker_hand_calculator.ipynb

First, let's import the necessary libraries:

In [ ]:
import json

Now, we define our test cases (in json string format)

In [137]:
# test cases:
# Tie
handOne_1 =  "[{\"value\":1,\"suit\":\"hearts\"},{\"value\":2,\"suit\":\"hearts\"},{\"value\":3,\"suit\":\"hearts\"},{\"value\":4,\"suit\":\"hearts\"},{\"value\":5,\"suit\":\"hearts\"}]"
handTwo_1 = "[{\"value\":1,\"suit\":\"hearts\"},{\"value\":2,\"suit\":\"hearts\"},{\"value\":3,\"suit\":\"hearts\"},{\"value\":4,\"suit\":\"hearts\"},{\"value\":5,\"suit\":\"hearts\"}]"

#handOne
handOne_2 = "[{\"value\":8,\"suit\":\"hearts\"},{\"value\":2,\"suit\":\"hearts\"},{\"value\":3,\"suit\":\"hearts\"},{\"value\":4,\"suit\":\"hearts\"},{\"value\":1,\"suit\":\"hearts\"}]"
handTwo_2 = "[{\"value\":1,\"suit\":\"hearts\"},{\"value\":2,\"suit\":\"hearts\"},{\"value\":3,\"suit\":\"hearts\"},{\"value\":4,\"suit\":\"hearts\"},{\"value\":5,\"suit\":\"spades\"}]"

#handOne
handOne_3 = "[{\"value\":8,\"suit\":\"hearts\"},{\"value\":2,\"suit\":\"hearts\"},{\"value\":3,\"suit\":\"hearts\"},{\"value\":4,\"suit\":\"hearts\"},{\"value\":1,\"suit\":\"hearts\"}]"
handTwo_3 = "[{\"value\":1,\"suit\":\"hearts\"},{\"value\":2,\"suit\":\"hearts\"},{\"value\":3,\"suit\":\"hearts\"},{\"value\":4,\"suit\":\"hearts\"},{\"value\":5,\"suit\":\"spades\"}]"

#handTwo
handTwo_4 = "[{\"value\":8,\"suit\":\"hearts\"},{\"value\":8,\"suit\":\"spades\"},{\"value\":8,\"suit\":\"diamonds\"},{\"value\":8,\"suit\":\"clubs\"},{\"value\":1,\"suit\":\"hearts\"}]"
handOne_4 = "[{\"value\":1,\"suit\":\"hearts\"},{\"value\":2,\"suit\":\"hearts\"},{\"value\":3,\"suit\":\"hearts\"},{\"value\":4,\"suit\":\"hearts\"},{\"value\":6,\"suit\":\"spades\"}]"

It might seem silly to have a one line loader function like this. However, this sort of thing is necessary for maintainability on a project with a large code base if the programmer doesn't want to spend eternity digging out 30 instance of str_ = json.loads(...) to replace it with some more complicated logic to handle the new cases the project manager suddenly decided the code should handle.

In [87]:
# loader function
def load_hand(hand_):
    return json.loads(hand_)

## List manipulation helper functions
The idea behind these functions is to have short, sweet, atomistic functions that do *one thing* and do it *exactly* like we expect it to be done.

One of the reasons to do this is, generally speaking, you want to debug your code *while* you're writing it, not after. In other words:
1. Write a small piece of code
2. Throw every relavent test case at it you can think of
3. Move on when the code works exactly the way you want it to work

The next three functions are examples of this -- low level list manipulation that we will leverage for more complicated functions later on.

If you don't already design your code this way, you should start -- you'll thank me later.

In [139]:
# return list of card values
def card_list(hand):
    cl = []
    for tmp in hand:
        cl.append(tmp['value'])
    return cl

In [19]:
#return a list of card suits
def suit_list(hand):
    sl = []
    for tmp in hand:
        sl.append(tmp['suit'])
    return sl

In [26]:
#return a list of card value counts (to compare quads vs. three of a kind, etc.)
def count_list(card_list):
    ct = []
    lst = set(card_list)
    for l in lst:
        tmp = card_list.count(l)
        ct.append(tmp)
    return ct

## Hand tests

Now, we can start testing for specific hands

In [131]:
# For a flush, we do the following
# 1. Get a list of suits
# 2. Get the unique suits in the hand (using the set function)
# 3. Check the length. For a flush, the number of unique suits will be 1 (all same suit), so the length of the set will be one
def testFlush(hand):
    out = suit_list(hand)
    tmp = set(out)
    if len(tmp) == 1:
        return 'Flush', True
    else:
        return '', False

In [119]:
# For a straight, we do the following:
# 1. Get the card values
# 2. Sort them
# 3. Get the set
# The difference between the high and low card should be == 4 and the length of the list should be 5 (no pairs)
def testStraight(hand):
    out = card_list(hand)
    out.sort()
#     print(out)
    tmp = set(out)
#     print(tmp)
    if (out[4] - out[0] == 4) and len(tmp) == 5:
        return 'Straight', True
    else:
        return '', False

In [53]:
# Check for pairs
def pair_check(hand):
    cl = card_list(hand)
    lcl = set(cl)
    ct = count_list(cl)
    if len(ct) == 5:
        return 'High'#High card only
    if len(ct) == 4:
        return 'Pair'#Pair
    if len(ct) == 3:
        if max(ct) == 2:
            return 'Two'#[2,2,1] -- Two of a kind
        if max(ct) == 3:
            return 'Three'#[3,1,1] -- Three of a kind
    if len(ct) == 2:
        if max(ct) == 3:
            return 'Full' #[3,2] Full house
        if max(ct) == 4:
            return 'Quad' #[4,1] Four of a kind

In [62]:
#If a hand is a straight and a flush, it's either a straight flush or a royal flush
def testStraightFlush(hand):
    _, tst1 = testStraight(hand)
    _, tst2 = testFlush(hand)
    out = card_list(hand)
    out.sort()
    if tst1 and tst2 and max(out) == 14:
        return 'Royal'
    if tst1 and tst2:
        return 'StFl'
    else:
        return []

## Get hand, hand value, main function

Below are two more helper functions -- one to get the hand string and the other to score the hand based on string -- and the main function

In [121]:
def get_hand(hand):
    tst = []
    tst = testStraightFlush(hand)
    if not tst:
#         print(1)
        tst, _ = testStraight(hand)
    if not tst:
#         print(2)
        tst, _ = testFlush(hand)
    if not tst:
#         print(3)
        tst = pair_check(hand)
    return tst

In [83]:
def value_hand(val_str):
    if val_str == 'Royal':
        return 1
    if val_str == 'StFl':
        return 2
    if val_str == 'Quad':
        return 3
    if val_str == 'Full':
        return 4
    if val_str == 'Flush':
        return 5
    if val_str == 'Straight':
        return 6
    if val_str == 'Three':
        return 7
    if val_str == 'Two':
        return 8
    if val_str == 'One':
        return 9
    if val_str == 'High':
        return 10

In [132]:
def solution(handOne: str, handTwo: str) -> str:
    
    handOne = load_hand(handOne)
    handTwo = load_hand(handTwo)
    
    hand1_str = get_hand(handOne)
    hand2_str = get_hand(handTwo)
    
    val1 = value_hand(hand1_str)
    val2 = value_hand(hand2_str)
    
    if val1 > val2:
        return 'handTwo'
    if val1 < val2:
        return 'handOne'
    if val1 == val2:
        return 'Tie'

## Test Cases

In [138]:
case1 = solution(handOne_1, handTwo_1)
print(case1)
case2 = solution(handOne_2, handTwo_2)
print(case2)
case3 = solution(handOne_3, handTwo_3)
print(case3)
case4 = solution(handOne_4, handTwo_4)
print(case4)

Tie
handOne
handOne
handTwo
